# 02 - ML Experimentation with Custom Model

The purpose of this notebook is to use [custom training](https://cloud.google.com/ai-platform-unified/docs/training/custom-training) to train a keras classifier to predict whether a given trip will result in a tip > 20%. The notebook covers the following tasks:
1. Preprocess the data locally using Apache Bean.
2. Train and test custom model locally using a Keras implementation.
3. Submit a Dataflow job to preprocess the data at scale.
4. Submit a custom training job to Vertex AI using a [pre-built container](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers).
5. Upload the trained model to Vertex AI.
6. Exract and visualize experiment parameters from [Vertex AI Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction).

We use [Vertex TensorBoard](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-overview) 
and [Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata/introduction) to  track, visualize, and compare ML experiments.

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2?

In [ ]:
import os
import time
import logging
from datetime import datetime
import numpy as np

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow.keras as keras

from src.common import features
from src.model_training import data, model, defaults, trainer, exporter
from src.utils import datasource_utils
from src.preprocessing import etl

logging.getLogger().setLevel(logging.INFO)

print(f"TensorFlow: {tf.__version__}")
print(f"TensorFlow Transform: {tft.__version__}")

In [ ]:
PROJECT = 'ksalama-cloudml'  # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us'  # Change to your bucket.
SERVICE_ACCOUNT = "sa-vertex-ai-mlops@ksalama-cloudml.iam.gserviceaccount.com"

DATASET_DISPLAY_NAME = 'chicago_taxi_tips_v1'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}_classifier_v1'

WORKSPACE = f'gs://{BUCKET}/ucaip_demo/chicago_taxi/experiments'
RAW_SCHEMA_LOCATION = 'src/raw_schema/schema.pbtxt'

TENSORBOARD_DISPLAY_NAME = f'tb-{PROJECT}'
EXPERIMENT_NAME = f'{DATASET_DISPLAY_NAME}-experiment'

In [ ]:
from src.utils.vertex_utils import VertexClient
vertex_client = VertexClient(PROJECT, REGION, BUCKET)

## Create Vertex TensorBoard Instance 

In [ ]:
!gcloud beta ai tensorboards create --display-name={TENSORBOARD_DISPLAY_NAME} \
  --project={PROJECT} --region={REGION}

In [ ]:
tensorboard_resource_name = vertex_client.get_tensorboard_by_display_name(TENSORBOARD_DISPLAY_NAME).name
print("TensorBoard resource name:", tensorboard_resource_name)

## Initialize Workspace

In [ ]:
REMOVE_WORKSPACE = False
if tf.io.gfile.exists(WORKSPACE) and REMOVE_WORKSPACE:
    print("Removing previous local workspace...")
    tf.io.gfile.rmtree(WORKSPACE)

if not tf.io.gfile.exists(WORKSPACE):
    print("Creating new local workspace...")
    tf.io.gfile.mkdir(WORKSPACE)

print("Workspace is ready.")

## Initialize Vertex AI Experiment

In [ ]:
vertex_client.set_experiment(EXPERIMENT_NAME)
run_id = f"run-local-{datetime.now().strftime('%Y%m%d%H%M%S')}"
_ = vertex_client.start_experiment_run(run_id)

EXPERIMENT_RUN_DIR = os.path.join(WORKSPACE, EXPERIMENT_NAME, run_id)
print("Experiment run directory:", EXPERIMENT_RUN_DIR)

## 1. Preprocess the data using Apache Beam

The Apache Beam pipeline of data preprocessing is implemented in the [preprocessing](src/preprocessing) directory.

In [ ]:
EXPORTED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'exported_data')
TRANSFORMED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'transformed_data')
TRANSFORM_ARTEFACTS_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'transform_artifacts')

### Get Source Query from Managed Dataset

In [ ]:
DATA_SPLIT = 'UNASSIGNED'
LIMIT = 5120

raw_data_query = datasource_utils.get_training_source_query(
    project=PROJECT, 
    region=REGION, 
    dataset_display_name=DATASET_DISPLAY_NAME, 
    data_split=DATA_SPLIT, 
    limit=LIMIT
)

print(raw_data_query)

### Test Data Preprocessing Locally

In [ ]:
args = {
    'runner': 'DirectRunner',
    'raw_data_query': raw_data_query,
    'write_raw_data': True,
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'transformed_data_prefix': TRANSFORMED_DATA_PREFIX,
    'transform_artefact_dir': TRANSFORM_ARTEFACTS_DIR,
    'temporary_dir': os.path.join(WORKSPACE, 'tmp'),
    'gcs_location': f'gs://{BUCKET}/bq_tmp',
    'project': PROJECT
}

In [ ]:
vertex_client.log_params(args)

In [ ]:
print("Data preprocessing started...")
etl.run_transform_pipeline(args)
print("Data preprocessing completed.")

In [ ]:
!gsutil ls {EXPERIMENT_RUN_DIR}

In [ ]:
vertex_client.log_params({"transformed_data_path": TRANSFORMED_DATA_PREFIX})
vertex_client.log_params({"transform_artifacts_dir": TRANSFORM_ARTEFACTS_DIR})

## 2. Train a Custom Model Localy using a Keras Implementation

The Keras implementation of the custom model is in the [model_training](src/model_training) directory.

In [ ]:
LOG_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'logs')
EXPORT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'model')

### Read transformed data

In [ ]:
tft_output = tft.TFTransformOutput(TRANSFORM_ARTEFACTS_DIR)
transform_feature_spec = tft_output.transformed_feature_spec()
transform_feature_spec

In [ ]:
train_data_file_pattern = os.path.join(TRANSFORMED_DATA_PREFIX,'train/data-*.gz')
eval_data_file_pattern = os.path.join(TRANSFORMED_DATA_PREFIX,'eval/data-*.gz')

for input_features, target in data.get_dataset(
    train_data_file_pattern, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} {input_features[key].dtype}: {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

### Create model inputs

In [ ]:
input_layers = model.create_model_inputs()
input_layers

### Create hyperparameters

In [ ]:
hyperparams = {
    "hidden_units": [64, 32]
}

hyperparams = defaults.update_hyperparams(hyperparams)
hyperparams

### Create and test model inputs and outputs

In [ ]:
classifier = model.create_binary_classifier(tft_output, hyperparams)
classifier.summary()

In [ ]:
keras.utils.plot_model(
    classifier, 
    show_shapes=True, 
    show_dtype=True
)

In [ ]:
classifier(input_features)

### Train the model locally.

In [ ]:
logging.getLogger().setLevel(logging.INFO)

hyperparams["learning_rate"] = 0.001
hyperparams["num_epochs"] = 5
hyperparams["batch_size"] = 512

vertex_client.log_params(hyperparams)

In [ ]:
classifier = trainer.train(
    train_data_dir=train_data_file_pattern,
    eval_data_dir=eval_data_file_pattern,
    tft_output_dir=TRANSFORM_ARTEFACTS_DIR,
    hyperparams=hyperparams,
    log_dir=LOG_DIR,
)

In [ ]:
val_loss, val_accuracy = trainer.evaluate(
    model=classifier,
    data_dir=eval_data_file_pattern,
    raw_schema_location=RAW_SCHEMA_LOCATION,
    tft_output_dir=TRANSFORM_ARTEFACTS_DIR,
    hyperparams=hyperparams,
)

In [ ]:
vertex_client.log_metrics(
    {"val_loss": val_loss, "val_accuracy": val_accuracy})

In [ ]:
!tb-gcp-uploader --tensorboard_resource_name={tensorboard_resource_name} \
  --logdir={LOG_DIR} \
  --experiment_name={EXPERIMENT_NAME} --one_shot=True

### Export the trained model

In [ ]:
saved_model_dir = os.path.join(EXPORT_DIR)

exporter.export_serving_model(
    classifier=classifier,
    serving_model_dir=saved_model_dir,
    raw_schema_location=RAW_SCHEMA_LOCATION,
    tft_output_dir=TRANSFORM_ARTEFACTS_DIR,
)

In [ ]:
!saved_model_cli show --dir={saved_model_dir} --tag_set=serve --signature_def=serving_tf_example

In [ ]:
!saved_model_cli show --dir={saved_model_dir} --tag_set=serve --signature_def=serving_default

### Test the exported SavedModel

In [ ]:
serving_model = tf.saved_model.load(saved_model_dir)

In [ ]:
file_names = tf.data.TFRecordDataset.list_files(EXPORTED_DATA_PREFIX + '-*.tfrecord')
for batch in tf.data.TFRecordDataset(file_names).batch(3).take(1):
    predictions = serving_model.signatures['serving_tf_example'](batch)
    for key in predictions:
        print(f"{key}: {predictions[key]}")

In [ ]:
import tensorflow_data_validation as tfdv
from tensorflow_transform.tf_metadata import schema_utils

raw_schema = tfdv.load_schema_text(RAW_SCHEMA_LOCATION)
raw_feature_spec = schema_utils.schema_as_feature_spec(raw_schema).feature_spec

In [ ]:
instance = {
    "dropoff_grid": "POINT(-87.6 41.9)",
    "euclidean": 2064.2696,
    "loc_cross": "",
    "payment_type": "Credit Card",
    "pickup_grid": "POINT(-87.6 41.9)",
    "trip_miles": 1.37,
    "trip_day": 12,
    "trip_hour": 6,
    "trip_month": 2,
    "trip_day_of_week": 4,
    "trip_seconds": 555,
}

for feature_name in instance:
    dtype = raw_feature_spec[feature_name].dtype
    instance[feature_name] = tf.constant([[instance[feature_name]]], dtype)

In [ ]:
predictions = serving_model.signatures['serving_default'](**instance)
for key in predictions:
    print(f"{key}: {predictions[key].numpy()}")

## Start a new Vertex AI Experiment Run

In [ ]:
vertex_client.set_experiment(EXPERIMENT_NAME)
run_id = f"run-gcp-{datetime.now().strftime('%Y%m%d%H%M%S')}"
_ = vertex_client.start_experiment_run(run_id)

EXPERIMENT_RUN_DIR = os.path.join(WORKSPACE, EXPERIMENT_NAME, run_id)
print("Experiment run directory:", EXPERIMENT_RUN_DIR)

## 3. Submit a Data Processing Job to Dataflow

In [ ]:
EXPORTED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'exported_data')
TRANSFORMED_DATA_PREFIX = os.path.join(EXPERIMENT_RUN_DIR, 'transformed_data')
TRANSFORM_ARTEFACTS_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'transform_artifacts')

In [ ]:
DATA_SPLIT = 'UNASSIGNED'
LIMIT = 1000000
raw_data_query = datasource_utils.get_training_source_query(
    project=PROJECT, 
    region=REGION, 
    dataset_display_name=DATASET_DISPLAY_NAME, 
    data_split=DATA_SPLIT, 
    limit=LIMIT
)

args = {
    'runner': 'DataflowRunner',
    'raw_data_query': raw_data_query,
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'transformed_data_prefix': TRANSFORMED_DATA_PREFIX,
    'transform_artefact_dir': TRANSFORM_ARTEFACTS_DIR,
    'write_raw_data': False,
    'temporary_dir': os.path.join(WORKSPACE, 'tmp'),
    'gcs_location': os.path.join(WORKSPACE, 'bq_tmp'),
    'project': PROJECT,
    'region': REGION,
    'setup_file': './setup.py'
}

In [ ]:
vertex_client.log_params(args)

In [ ]:
logging.getLogger().setLevel(logging.ERROR)

print("Data preprocessing started...")
etl.run_transform_pipeline(args)
print("Data preprocessing completed.")

In [ ]:
!gsutil ls {EXPERIMENT_RUN_DIR}

In [ ]:
vertex_client.log_params({"transformed_data_path": TRANSFORMED_DATA_PREFIX})
vertex_client.log_params({"transform_artifacts_dir": TRANSFORM_ARTEFACTS_DIR})

## 4. Submit a Custom Training Job to Vertex AI

In [ ]:
LOG_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'logs')
EXPORT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'model')

### Test the training task locally

In [ ]:
!python -m src.model_training.task \
    --model-dir={EXPORT_DIR} \
    --log-dir={LOG_DIR} \
    --train-data-dir={TRANSFORMED_DATA_PREFIX}/train/* \
    --eval-data-dir={TRANSFORMED_DATA_PREFIX}/eval/*  \
    --tft-output-dir={TRANSFORM_ARTEFACTS_DIR} \
    --num-epochs=5 \
    --hidden-units=32,32 \
    --experiment-name={EXPERIMENT_NAME} \
    --run-name={run_id} \
    --project={PROJECT} \
    --region={REGION} \
    --staging-bucket={BUCKET}

### Prepare training package

In [ ]:
TRAINER_PACKAGE_DIR = os.path.join(WORKSPACE, 'trainer_packages')

In [ ]:
!rm -r src/__pycache__/
!rm -r src/.ipynb_checkpoints/
!rm -r src/raw_schema/.ipynb_checkpoints/
!rm -f custom_job.tar custom_job.tar.gz

!mkdir custom_job

!cp setup.py custom_job/
!cp -r src custom_job/
!tar cvf custom_job.tar custom_job
!gzip custom_job.tar
!gsutil cp custom_job.tar.gz {TRAINER_PACKAGE_DIR}/
!rm -r custom_job
!rm -r custom_job.tar.gz

### Prepare the training job

In [ ]:
TRAIN_RUNTIME = 'tf-cpu.2-4'
TRAIN_IMAGE = f"gcr.io/cloud-aiplatform/training/{TRAIN_RUNTIME}:latest"

In [ ]:
num_epochs = 10
learning_rate = 0.001
hidden_units = "64,64"

trainer_args = [
    f'--train-data-dir={TRANSFORMED_DATA_PREFIX + "/train/*"}',
    f'--eval-data-dir={TRANSFORMED_DATA_PREFIX + "/eval/*"}',
    f'--tft-output-dir={TRANSFORM_ARTEFACTS_DIR}',
    f'--num-epochs={num_epochs}',
    f'--learning-rate={learning_rate}',
    f'--project={PROJECT}',
    f'--region={REGION}',
    f'--staging-bucket={BUCKET}',
    f'--experiment-name={EXPERIMENT_NAME}'
]

In [ ]:
training_spec = [
    {
        "replica_count": 1,
        "machine_spec": {
            "machine_type": 'n1-standard-4',
            "accelerator_count": 0
    },
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [os.path.join(TRAINER_PACKAGE_DIR, 'custom_job.tar.gz')],
            "python_module": "src.model_training.task",
            "args": trainer_args,
        }
    }
]

### Submit the training job

In [ ]:
print("Submitting a custom training job...")
job = vertex_client.submit_custom_job(
    training_spec=training_spec,
    experiment_dir=EXPERIMENT_RUN_DIR,
    service_account=SERVICE_ACCOUNT,
    tensorboard_resource_name=tensorboard_resource_name
)
print(f"Job {job.name} sbumitted.")

### Monitor job state

In [ ]:
while True:
    response = vertex_client.get_custom_job_by_uri(job.name)
    if response.state.name == 'JOB_STATE_SUCCEEDED':
        print("Training job completed. - Training Time:", response.update_time - response.create_time)
        break
    elif response.state.name == 'JOB_STATE_FAILED':
        print("Training job failed!")
        break
    else:
        print(f"Training job state is: {response.state.name}.")
    time.sleep(60)

## 5. Upload exported model to Vertex AI Models

In [ ]:
exported_model_dir = os.path.join(TRAINING_DIR, 'model')
!gsutil ls {exported_model_dir}

In [ ]:
SERVING_RUNTIME='tf2-cpu.2-4'
SERVING_IMAGE = f"gcr.io/cloud-aiplatform/prediction/{SERVING_RUNTIME}:latest"

In [ ]:
vertex_model = vertex_client.upload_model(
    display_name=MODEL_DISPLAY_NAME,
    model_artifact_uri=exported_model_dir,
    serving_image_uri=SERVING_IMAGE,
)

In [ ]:
vertex_model.gca_resource

## 6. Exract and Visualize Experiment Parameters

In [ ]:
experiment_df = vertex_client.get_experiment_df(EXPERIMENT_NAME)
experiment_df

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams["figure.figsize"] = [15, 5]

ax = pd.plotting.parallel_coordinates(
    experiment_df.reset_index(level=0),
    "run_name",
    cols=[
        "param.num_epochs",
        "param.hidden_units",
        "param.learning_rate",
        "metric.val_loss",
        "metric.val_accuraccy",
    ],
)
ax.set_yscale("symlog")
ax.legend(bbox_to_anchor=(1.0, 0.5))

In [ ]:
print("Vertex AI Experiments:")
print(
    f"https://console.cloud.google.com/vertex-ai/locations{REGION}/experiments/{EXPERIMENT_NAME}/metrics?project={PROJECT}"
)